<a href="https://colab.research.google.com/github/gibsonx/AnsibleGUI/blob/master/yolov5_leaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics >> /dev/null
!pip install yolov5 >> /dev/null
!pip install supervision >> /dev/null

In [18]:
import supervision as sv
import numpy as np
from ultralytics import YOLO
import yolov5
import cv2

In [13]:
VIDEO_PATH = "/content/drive/MyDrive/Dataset/water_leaf/swimming_pool.mp4"
# model = YOLO("/content/drive/MyDrive/Dataset/water_leaf/bestpla.pt")
model = yolov5.load('/content/drive/MyDrive/Dataset/water_leaf/bestpla.pt')
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

In [17]:
tracker = sv.ByteTrack()
box_annotator = sv.BoxAnnotator()
ellipse_annotator = sv.EllipseAnnotator()
label_annotator = sv.LabelAnnotator()
triangle_annotator = sv.TriangleAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_yolov5(results)
    detections = tracker.update_with_detections(detections)

    # labels = [
    #     f"#{tracker_id} {results.names[class_id]}"
    #     for class_id, tracker_id
    #     in zip(detections.class_id, detections.tracker_id)
    # ]


    annotated_frame = ellipse_annotator.annotate(
        scene=frame.copy(),
        detections=detections
    )

    annotated_frame = triangle_annotator.annotate(
        scene=annotated_frame.copy(),
        detections=detections
    )

    return label_annotator.annotate(
        annotated_frame, detections=detections)

sv.process_video(
    source_path=VIDEO_PATH,
    target_path="result.mp4",
    callback=callback
)

TypeError: 'Detections' object is not subscriptable